In [13]:
test_file = "../data/COLIEE2024statute_data-English/train/riteval_R04_en.xml"
dev_file = "../data/COLIEE2024statute_data-English/train/riteval_R03_en.xml"

In [3]:
ls ../data/COLIEE2024statute_data-English/train

riteval_H18_en.xml*  riteval_H24_en.xml*  riteval_H30_en.xml*
riteval_H19_en.xml*  riteval_H25_en.xml*  riteval_R01_en.xml*
riteval_H20_en.xml*  riteval_H26_en.xml*  riteval_R02_en.xml*
riteval_H21_en.xml*  riteval_H27_en.xml*  riteval_R03_en.xml*
riteval_H22_en.xml*  riteval_H28_en.xml*  riteval_R04_en.xml*
riteval_H23_en.xml*  riteval_H29_en.xml*


In [4]:
from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

In [5]:
from bs4 import BeautifulSoup
import re
import json

def get_article(articles):
    result = {}
    current_statue = "(non-statute)"
    for i in re.split(r"(.*)", articles.strip()):
        if len(i) == 0 or i == "\n":
            continue
        if re.search(r"^\(.*\)$", i):
            current_statue = i.strip()
            if current_statue not in result:
                result.update({current_statue: []})
        else:
            if current_statue not in result:
                result.update({current_statue: []})
            result[current_statue].append(i)
    return result

def build_test(filename):
    result = {}
    with open(filename, 'r') as f:
        data = f.read()

    data = BeautifulSoup(data, "xml").find_all('pair')
    for i in data:
        id = i.get('id')
        result.update({id: {}})
        result[id].update({"label": i.get('label')})
        articles = i.find('t1').text.strip()
        # articles = get_article(articles)
        result[id].update({"result": articles})
        result[id].update({"content": i.find('t2').text.strip()})
    return result

def write_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [6]:
import xml.etree.ElementTree as Et
import glob

def format_first_line(text):
    lines = text.split("\n")
    results = []
    for line in lines:
        if line[0] == "":
            continue
        if line[0] == "(" and line[-1] == ")":
            continue
        results.append(line)
    return "\n".join(results)

def load_samples(filexml):
    # try:
        tree = Et.parse(filexml)
        root = tree.getroot()
        samples = []
        for i in range(0, len(root)):
            sample = {'result': []}
            for j, e in enumerate(root[i]):
                if e.tag == "t1":
                    sample['result'] = format_first_line(e.text.strip())
                elif e.tag == "t2":
                    question = e.text.strip()
                    sample['content'] = question if len(question) > 0 else None
            sample.update(
                {'index': root[i].attrib['id'], 'label': root[i].attrib.get('label', "N")})
            # filter the noise samples
            if sample['content'] is not None:
                samples.append(sample)
            else:
                print("[Important warning] samples {} is ignored".format(sample))
        return samples

def load_test_data_samples(path_folder_base, test_id):
    data = []
    test = load_samples(f"{path_folder_base}/riteval_{test_id}.xml")
    for file_path in glob.glob(f"{path_folder_base}/riteval_{test_id}.xml"):
        data = data + load_samples(file_path)
    return data


def load_all_data_samples(path_folder_base):
    data = []
    for file_path in glob.glob("{}/*.xml".format(path_folder_base)):
        data = data + load_samples(file_path)
    return data

def check_false_labels(pred, false_labels):
	for label in false_labels:
		if label in pred:
			return True
	return False

def prompting(prompt="{{premise}}\nBased on the previous passage, {{hypothesis}}? ", data=None):
    return prompt.replace("{{premise}}", data['result']).replace('{{hypothesis}}', data['content'])

In [7]:
filenames = get_all_files_from_path("../data/COLIEE2024statute_data-English/train")

for file in filenames:
    data = build_test(file)
    saved_file = file.split("/")[-1][:-3]+"json"
    write_json("../data/json/train/"+saved_file, data)

# Evaluate

In [8]:
data = load_samples(test_file)
# data = build_test(test_file)
# result = []
# for k in data:
#     result.append(data[k])

# data = result
data

[{'result': "Article 5 (1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.\n(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.\nArticle 6 (1) A minor who is permitted to conduct one or multiple types of business has the same capacity to act as an adult as far as that business is concerned.",
  'content': 'A juridical act by a minor who is permitted to conduct business may not be rescinded even if the judicial act does not concern the business.',
  'index': 'R04-01-I',
  'label': 'N'},
 {'result': "Article 5 (1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.\n(2) A juridical act in contravention

## Load Model

In [9]:
from transformers import AutoTokenizer, BloomForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "declare-lab/flan-alpaca-xxl"
cache_dir = "/home/congnguyen/drive/.cache"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(
		model_name, device_map="auto", cache_dir=cache_dir, torch_dtype=torch.float16, load_in_8bit=True
	)

/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████| 5/5 [02:55<00:00, 35.14s/it]


## Inference

In [10]:
import re
from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()
    
prompt_template = "{{premise}}\nBased on the previous passage, {{hypothesis}}? "

result = []
count = 0
for item in tqdm(data):
    label = item["label"]
    text = prompting(prompt_template, item)
    print(text)
    inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    outputs = model.generate(inputs, max_new_tokens=256)
    output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
    print(output_text)
    if "yes" in output_text:
        output_text = "Y"
    else:
        output_text = "N"
    if output_text == label:
        count+=1

  0%|                                                                                             | 0/101 [00:00<?, ?it/s]

Article 5 (1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
Article 6 (1) A minor who is permitted to conduct one or multiple types of business has the same capacity to act as an adult as far as that business is concerned.
Based on the previous passage, A juridical act by a minor who is permitted to conduct business may not be rescinded even if the judicial act does not concern the business.? 


  1%|▊                                                                                    | 1/101 [00:05<08:40,  5.20s/it]

no, the judicial act by a minor who is permitted to conduct business may not be rescinded even if the judicial act does not concern the business.
Article 5 (1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
Article 120 (1) An act that is voidable on the grounds of the qualified legal capacity to act of the person who did the act may be rescinded only by the person with qualified legal capacity (in the case of an act performed by the person as a legal representative of another person with limited capacity, including that other person with limited capacity), or an agent or successor thereof, or a person who has the authority to give consent thereto.
Based on the previous passage, When a minor concludes a contract withou

  2%|█▋                                                                                   | 2/101 [00:05<03:53,  2.36s/it]

...

no, the court may grant a reasonable period of time for the reimbursement only if the possessor was in bad faith.
Article 354 If the claim of a pledgee of movables is not satisfied, the pledgee may make a request to the court seeking the immediate appropriation of the thing pledged for the satisfaction of that claim in accordance with the evaluation of an appraiser only when there are reasonable grounds.In this case, the pledgee of movables must notify the obligor in advance of the request.
Based on the previous passage, If the claim of a pledgee of movables is not satisfied, the pledgee may immediately appropriate the thing pledged for the satisfaction of that claim by getting the permission of the court, instead of getting the evaluation of an appraiser, only when there are reasonable grounds for not getting the evaluation of an appraiser.? 


 99%|██████████████████████████████████████████████████████████████████████████████████▏| 100/101 [02:34<00:02,  2.47s/it]

no, the pledgee may not immediately appropriate the thing pledged for the satisfaction of that claim by getting the permission of the court, instead of getting the evaluation of an appraiser.
Article 494(1) In the following cases, a performer may deposit the subject matter of the performance with an official depository for the benefit of the obligee. In such a case, the claim is extinguished when the performer makes the deposit:
(i) the performer tenders the performance, and the obligee refuses to accept it; 
Article 497 In the following cases, the performer may, with the permission of the court, sell the thing that is the subject matter of performance at public auction and deposit the proceeds of such sales with an official depository:
(iii) excessive expenses are required for the preservation of the thing; or
Based on the previous passage, A performer tendered the performance, and the obligee refused to accept it. In this case, if excessive expenses are required for the preservation 

100%|███████████████████████████████████████████████████████████████████████████████████| 101/101 [02:34<00:00,  1.53s/it]

yes


In [11]:
count

76

# Test

In [35]:
test_label = []
for item in data:
    test_label.append(item['label'])

count = 0
for r, label in zip(result, test_label):
    check = "N"
    if "yes" in r[3:]:
        check = "Y"
    if check == label:
        count += 1

In [36]:
count/len(test_label)

0.5148514851485149

In [40]:
count / len(test_label)

0.801980198019802